<a href="https://colab.research.google.com/github/Misudhari/Deep-Learning-Lab/blob/main/HyperParameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SyntaxError('GPU Device Not Found')
print('Found GPU at:{}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at:/device:GPU:0


In [17]:
import pandas as pd
from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split

In [26]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/Tetuan City power consumption.csv')
# print(data.head())
x = data.iloc[:,:5]
y = data['Zone 1 Power Consumption']

X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.3, random_state = 0)

In [27]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers', 2, 20)):
    model.add(layers.Dense(units = hp.Int('units_' + str(i),
                                          min_value=32,
                                          max_value=512,
                                          step=32),
                           activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
         loss='mean_absolute_error',
         metrics=['mean_absolute_error'])
    return model
    

In [28]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name = 'Power Cosumtion'
)

INFO:tensorflow:Reloading Oracle from existing project project/Power Cosumtion/oracle.json


INFO:tensorflow:Reloading Oracle from existing project project/Power Cosumtion/oracle.json


In [29]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [30]:
tuner.search(X_train, Y_train, epochs=5, validation_data=(X_test, Y_test))

Trial 5 Complete [00h 01m 02s]
val_mean_absolute_error: 5932.274576822917

Best val_mean_absolute_error So Far: 5374.728678385417
Total elapsed time: 00h 05m 29s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [31]:
tuner.results_summary()

Results summary
Results in project/Power Cosumtion
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 18
units_0: 416
learning_rate: 0.01
Score: 5374.728678385417
Trial summary
Hyperparameters:
num_layers: 10
units_0: 416
learning_rate: 0.01
Score: 5384.830403645833
Trial summary
Hyperparameters:
num_layers: 10
units_0: 288
learning_rate: 0.01
Score: 5428.66552734375
Trial summary
Hyperparameters:
num_layers: 17
units_0: 448
learning_rate: 0.001
Score: 5731.8603515625
Trial summary
Hyperparameters:
num_layers: 5
units_0: 192
learning_rate: 0.001
Score: 5932.274576822917
